# Backpropagation Lab





In [ ]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1 Avoiding Overfit: Early Stopping and Loss Regularization

### 1.1 (10%) No overfit avoidance
Train the sklearn [MLP classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier) on the [Iris Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff).  Use 3 output nodes (1 per class). Expanding the one output variable into 3 is called one-hot encoding or dummy variable encoding. There are lots of ways to implement this including the Pandas get_dummies method. This experiment is set up to run a little longer to better see the effects of overfit.  Be patient as there are lots of hidden nodes and a high max iterations setting.

Use default parameters except for the following:
- hidden_layer_sizes = [64] - One hidden layer with 64 hidden nodes
- activation = 'logistic'
- solver = 'sgd'
- alpha = 0
- batch_size = 1
- learning_rate_init = 0.01
- shuffle = True
- momentum = 0
- n_iter_no_change = 50
- max_iterations = 10000

Use a random 80/20 split of the data.  Run it a few times with different random training/test splits and give average values for
- Number of iterations until convergence
- Training set accuracy
- Test set accuracy
For one run observe the softmax probabilities on the test set using clf.predict_proba

In [2]:
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff

% 1. Title: Iris Plants Database
% 
% 2. Sources:
%      (a) Creator: R.A. Fisher
%      (b) Donor: Michael Marshall (MARSHALL%PLU@io.arc.nasa.gov)
%      (c) Date: July, 1988
% 
% 3. Past Usage:
%    - Publications: too many to mention!!!  Here are a few.
%    1. Fisher,R.A. "The use of multiple measurements in taxonomic problems"
%       Annual Eugenics, 7, Part II, 179-188 (1936); also in "Contributions
%       to Mathematical Statistics" (John Wiley, NY, 1950).
%    2. Duda,R.O., & Hart,P.E. (1973) Pattern Classification and Scene Analysis.
%       (Q327.D83) John Wiley & Sons.  ISBN 0-471-22361-1.  See page 218.
%    3. Dasarathy, B.V. (1980) "Nosing Around the Neighborhood: A New System
%       Structure and Classification Rule for Recognition in Partially Exposed
%       Environments".  IEEE Transactions on Pattern Analysis and Machine
%       Intelligence, Vol. PAMI-2, No. 1, 67-71.
%       -- Results:
%          -- very low misclassification rates (0% for the setosa class)
%  

In [9]:
import pandas as pd 
from scipy.io import arff
import io
import requests
import sklearn
from sklearn.preprocessing import OneHotEncoder

url = "https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff"
response = requests.get(url)
data, meta = arff.loadarff(io.StringIO(response.text))

df = pd.DataFrame(data)
df = df.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df.head()

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
X = df.drop('class',axis=1).values
y = df['class'].values




In [3]:
#Iris with no regularization
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = iris.drop('class', axis=1).values
y = iris['class'].values

# One-hot encode the target
lb = LabelBinarizer()
y_onehot = lb.fit_transform(y)

results = []
for seed in [0, 1, 2]:
    X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=seed)
    clf = MLPClassifier(hidden_layer_sizes=[64], activation='logistic', solver='sgd',
                        alpha=0, batch_size=1, learning_rate_init=0.01,
                        shuffle=True, momentum=0, n_iter_no_change=50,
                        max_iter=10000, random_state=seed)
    clf.fit(X_train, y_train)
    train_acc = accuracy_score(np.argmax(y_train, axis=1), np.argmax(clf.predict(X_train), axis=1))
    test_acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(clf.predict(X_test), axis=1))
    n_iter = clf.n_iter_
    results.append((n_iter, train_acc, test_acc))
    if seed == 0:
        softmax_probs = clf.predict_proba(X_test)
        print(softmax_probs[:10])  # view first 10 test predictions

# Calculate averages
avg_iters = np.mean([r[0] for r in results])
avg_train_acc = np.mean([r[1] for r in results])
avg_test_acc = np.mean([r[2] for r in results])
print("Average Iterations:", avg_iters)
print("Average Training Accuracy:", avg_train_acc)
print("Average Test Accuracy:", avg_test_acc)

NameError: name 'iris' is not defined

#### Discussion
In the markdown cell below, write about your experience training the MLP classifier. Things to cover in your discussion: How long did it take? What was your accuracy on the training set? What was your accuracy on the testing set? Do you see any evidence of overfitting? If so, what could be done to reduce the effects of overfitting?

Writing this didn't take too long, I've had a few classes where we had to use models before so this was a bit of a review, especially using the iris set. Running this model didn't take long, at most a few seconds to run. On the training set, I got an accuarcy of 94% and on the test set I got 95%. Because these two accuracies are so close and we don't see a dramtic change from training to test, I don't think that there is any major overfitting in this model. 

### 1.2 (10%) Early Stopping (Validation Set)

- Do the same as above but this time with early stopping
- Use a validation set taken from the training set for your stopping criteria. Using 10-15% of the training set for a validation set is common. You do this simply by setting the MLPClassifier early_stopping, validation_fraction, and n_iter_no_change parameters.
- Run it a few times with different training/test splits and give average values for
    - Number of iterations until convergence
    - Training set accuracy
    - Test set accuracy
    - Best validation score (MLPClassifer attribute best_validation_score_)
- For one run create a graph with validation set accuracy (*y*-axis) vs epochs (*x*-axis). Hint: MLPClassifer attribute validation_scores_

Note: Due to the simplicity of and lack of noise in the iris data set you will not see the accuracy improvements that early stopping or loss regularization can give for more complex noisy datasets.  In particular, early stopping will have lower than expected results because with a very small VS taken from a very small training set there is less data to train on and more variance with the VS score.  Thus, you will probably get lower accuracies for VS than normal training for this less typical case.  But at least you will get practice on using early stopping and loss regularization for future data sets.

In [ ]:
#Iris with early stopping and validation scores graph
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

X = iris.drop('class', axis=1).values
y = iris['class'].values

lb = LabelBinarizer()
y_onehot = lb.fit_transform(y)

results = []
for seed in [0, 1, 2]:
    X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=seed)
    clf = MLPClassifier(hidden_layer_sizes=[64], activation='logistic', solver='sgd',
                        alpha=0, batch_size=1, learning_rate_init=0.01,
                        shuffle=True, momentum=0, n_iter_no_change=50,
                        max_iter=10000, random_state=seed,
                        early_stopping=True, validation_fraction=0.1)
    clf.fit(X_train, y_train)
    train_acc = accuracy_score(np.argmax(y_train, axis=1), np.argmax(clf.predict(X_train), axis=1))
    test_acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(clf.predict(X_test), axis=1))
    n_iter = clf.n_iter_
    best_val_score = clf.best_validation_score_
    results.append((n_iter, train_acc, test_acc, best_val_score))
    if seed == 0:
        softmax_probs = clf.predict_proba(X_test)
        print(softmax_probs[:10])  # view first 10 test predictions
        print("Validation scores per epoch:", clf.validation_scores_)

# Calculate averages
avg_iters = np.mean([r[0] for r in results])
avg_train_acc = np.mean([r[1] for r in results])
avg_test_acc = np.mean([r[2] for r in results])
avg_best_val = np.mean([r[3] for r in results])
print("Average Iterations:", avg_iters)
print("Average Training Accuracy:", avg_train_acc)
print("Average Test Accuracy:", avg_test_acc)
print("Average Best Validation Score:", avg_best_val)


plt.plot(clf.validation_scores_)
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.title('MLP Validation Accuracy per Epoch (Early Stopping)')
plt.show()

#### Discussion
Report on the average number of iterations until convergence. What is the effect of the use of the validation set and early stopping on training set accuracy and testing set accuracy? What do you attribute the performance change to? What do you see in the validation scores graph?

** Your discussion of average values above and the validation score graph **

### 1.3 (10%) Loss Regularization

- Do the same as in 1.1 but his time with loss regularization (Do not do early stopping)
- Run it with different L2 regularization parameter values (alpha).  The default for alpha is .0001.  Try other values such as .1, .01, .001, .00001, etc. Make a table with each row including:
    - The regularization parameter value
    - Number of iterations until convergence
    - Training set accuracy
    - Test set accuracy
    - Best loss value (MLPClassifer attribute best_loss_)
- Which regularization value gave you the best results?
- For your best regularization value do one run and create a graph with loss (*y*-axis) vs epochs (*x*-axis) for the training set (Hint: MLPClassifer attribute loss_curve_)

In [ ]:
#Iris with Loss Regularization 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

X = iris.drop('class', axis=1).values
y = iris['class'].values
lb = LabelBinarizer()
y_onehot = lb.fit_transform(y)

alpha_values = [0.1, 0.01, 0.001, 0.0001, 0.00001]
results = []
loss_curves = {}

for alpha in alpha_values:
    n_iters, train_accs, test_accs, best_losses = [], [], [], []
    for seed in [0, 1, 2]:
        X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=seed)
        clf = MLPClassifier(hidden_layer_sizes=[64], activation='logistic', solver='sgd',
                            alpha=alpha, batch_size=1, learning_rate_init=0.01,
                            shuffle=True, momentum=0, n_iter_no_change=50,
                            max_iter=10000, random_state=seed)
        clf.fit(X_train, y_train)
        train_acc = accuracy_score(np.argmax(y_train, axis=1), np.argmax(clf.predict(X_train), axis=1))
        test_acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(clf.predict(X_test), axis=1))
        n_iter = clf.n_iter_
        best_loss = clf.best_loss_
        n_iters.append(n_iter)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        best_losses.append(best_loss)
        if seed == 0:
            loss_curves[alpha] = clf.loss_curve_
    # Average results for this alpha
    results.append([alpha, np.mean(n_iters), np.mean(train_accs), np.mean(test_accs), np.mean(best_losses)])

# Print results table
print("Alpha | Iterations | Train Acc | Test Acc")
for row in results:
    print("{:.5f} | {:.1f} | {:.3f} | {:.3f} | {:.5f}".format(*row))

# Find best alpha (highest test accuracy)
best_row = max(results, key=lambda x: x[3])
best_alpha = best_row[0]
print("Best alpha:", best_alpha)

# For best alpha, plot loss curve for one run
import matplotlib.pyplot as plt
plt.plot(loss_curves[best_alpha])
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title(f'Loss Curve (alpha={best_alpha})')
plt.show()


#### Discussion
Write about which regularization method gave you the best results and why you think that happened. Also compare: no regularization, early stopping, and L2 loss regularization.

** Your discussion **

## 2 Hyperparameters 
In this section we use the [Vowel Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/vowel.arff) to consider the hyperparameters of learning rate, number of hidden nodes, and momentum. 

### 2.1 (10%) Vowel Dataset Questions
- Give the baseline accuracies for the Iris and Vowel datasets. Baseline accuracy is what you would get if the model just outputs the majority class of the data set (i.e. the output value which occurs most often). These two data sets are not great examples for this as they have an equal amount of each class, which is not typical.
- Discuss why the vowel data set will probably have lower accuracy than Iris.
- Consider which of the vowel dataset's input features you should not use in training and discuss why.

In [ ]:
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/vowel.arff

In [ ]:
url = 'https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/vowel.arff'
with urllib.request.urlopen(url, context=ssl_context) as response:
    data = response.read().decode('utf-8')

lines = data.strip().split('\n')
data_section = False
data_lines = []
for line in lines:
    if line.strip().lower() == '@data':
        data_section = True
        continue
    if data_section and line.strip() and not line.strip().startswith('%'):
        data_lines.append(line.strip())

vowel_cols = [
    'TrainOrTest','Speaker','Sex','Feature0','Feature1','Feature2','Feature3','Feature4',
    'Feature5','Feature6','Feature7','Feature8','Feature9','Class'
]
vowel = pd.read_csv(StringIO('\n'.join(data_lines)), header=None, names=vowel_cols)


In [ ]:
iris_baseline_acc = iris['class'].value_counts(normalize=True).max()
print("Iris Baseline Accuracy:", iris_baseline_acc)

In [ ]:
vowel_baseline_acc = vowel['Class'].value_counts(normalize=True).max()
print("Vowel Baseline Accuracy:", vowel_baseline_acc)

** Discuss these items here **

### 2.2 (10%) Learning Rate
Load the [Vowel Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/vowel.arff). Drop any features which you explained above as being inappropriate for training.

Hints: Consider the Pandas drop method for dropping columns. When you want to transform features in your data set there are lots of approaches. You could edit the arff file directly, or make the transforms in your code.  The Pandas replace method is nice for that. For example, if you wanted to change the vowel data set gender feature in a Pandas dataframe to 0/1 you could do the following:

vowel_df['Sex'] = vowel_df['Sex'].str.decode('utf-8')   //Changes the byte code data into a normal string, b'Male' becomes "Male"\
vowel_df = vowel_df.replace('Male', 0)\
vowel_df = vowel_df.replace('Female', 1)

- Use one layer of hidden nodes with the number of hidden nodes being twice the number of inputs.
- Use a random 75/25 split of the data for the training/test set.
- Do not use early stopping.
- Try at least 5 different learning rates (LR) from very small (e.g. .001) to pretty big (e.g. 10). Each LR will require a different number of epochs to learn. LR effects both accuracy and time required for learning.
- Create a table which includes a row for each LR.  Your table columns should be LR, # epochs to learn the model, final training set accuracy and final test set accuracy.  As learning rates get smaller, it usually takes more epochs to learn. If your model is stopping learning too soon (converging) by hitting max_iterations (in this case and in experiments below), then you need to increase your max_iterations parameter in order to give your model more learning time.  To keep things faster, you don't need to increase max_iter past 1000 if you don't want to, but point out when more iterations may have given improvement.

In real testing one averages the results of multiple trials per LR (and other parameters) with different intitial conditions (training/test split, initial weights, etc.). That gives more accurate results but is not required for this lab.

In [ ]:
# Train with different learning rates

#### Discussion
Discuss your table and the effect of different learning rates on both training time and accuracy

** Your discussion goes here **

### 2.3 (10%) Number of Hidden Nodes

Using the best LR you discovered, experiment with different numbers of hidden nodes.

- Start with 1 hidden node, then 2, and then double them for each test until you get no more improvement in accuracy. 
- Create a table just like above, except with # of hidden nodes rather than LR.

In general, whenever you are testing a parameter such as # of hidden nodes, keep testing values until no more improvement is found. For example, if 20 hidden nodes did better than 10, you would not stop at 20, but would try 40, etc., until you no longer got improvement.

In [ ]:
# Train with different numbers of hidden nodes

#### Discussion
Discuss your table and the effect of different numbers of hidden nodes on both training time and accuracy

** Your discussion goes here **

### 2.4 (10%) Momentum

Try at least 5 different momentum terms between 0 and just less than 1 using the best number of hidden nodes and LR from your earlier experiments.

- Create a table just like above, except with momentum values rather than LR or number of hidden nodes.

In [ ]:
# Train with different momentum values

#### Discussion
Discuss your table and the effect of momentum on both training time and accuracy

** Your discussion goes here **

### 2.5 (10%) Automatic Hyperparameter Discovery
Using the vowel dataset, automatically adjust the LR, # of hidden nodes, and momentum using [grid and random search](https://scikit-learn.org/stable/modules/grid_search.html) 
- For grid search include the most promising hyperparameter values you used in your experiments above.  You may add others also.
- Be patient as the grid search can take a while since it has to train all combinations of models. Don't use too many parameter options or it will be too slow.
- Report your best hyperparameters and accuracy.  Unfortunately, you will not always get as high a score as you might expect.  This is in part due to the simplicity of the dataset.  It also teaches that in gerneral you should not blindly assume that a tool will get you the results you expect, and that you may need to consider multiple approaches.

In [ ]:
#Grid search for hyperparameters.
#Here is one variation of code you could use for your grid search. You can try your own variation if you prefer.

from sklearn.model_selection import GridSearchCV
clf = MLPClassifier(activation='logistic', solver='sgd',alpha=0,early_stopping=True, n_iter_no_change=10, batch_size=1)
parameters = {'learning_rate_init':( .001, ?, ....), #You have to fill in the rest of your values for these lists
              'hidden_layer_sizes': ([8], [?], ...),
              'momentum':(0, ?, ...)
grid = GridSearchCV(clf, parameters)
grid.fit(X,Y)    #This takes a while to run
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
#Randomized search for hyperparameters
#Here is one variation of code you could use for your randomized search.

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
clf = MLPClassifier(activation='logistic', solver='sgd',alpha=0,early_stopping=True, n_iter_no_change=10, batch_size=1)
distributions = dict(learning_rate_init=uniform(loc=?, scale=?), #loc is the min val, and loc + scale is the max val
                    hidden_layer_sizes = ([8], [?], ...), #since there is no distribution it samples these values uniformly
                    momentum=uniform(loc=0,scale =.99))
search = RandomizedSearchCV(clf, distributions, n_iter=10)
search.fit(X,Y)
print(search.best_params_)
print(search.best_score_)

#### Discussion
Write about and compare grid and randomized parameter search. How much time do they take? Which is more efficient or accurate? When should you use each?

** Your discussion goes here **

## 3 Regression with MLPs

### 3.1 (10%) - Learn a regression data set of your choice

Train MLP on any real world data set that requires regression (i.e. has a real valued ouput) and discuss your effort and results.  While the [Irvine ML Repository](https://archive.ics.uci.edu) is a great resource, also onsider [Kaggle](https://www.kaggle.com) and [OpenML](https://openml.org) as other great places to find datasets.
- Use [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor) rather than MLPclassifier.  It has almost the exact same setup as MLPClassier except it uses the linear activation function for the output nodes and SSE as the loss function.  MLPClassier uses softmax activation for the output nodes and cross-entropy for the loss function.
- Use any reasonable hyperparameters that you want.  
- You will probably need to normalize input features.
- It is not typically necessary to normalize the output.
- Split into train and test and report the training and test set MAEs (Mean Absolute Error). For regression problems where we don't normalize the output, MAE is an intuitive measure as it shows exactly how much our output is off on average.

In [ ]:
# Load and Learn a real world regression data set
# To calculate MAE you could do a variation of the following

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(clf.predict(X_test), y_test))

#### Discussion
Report on your choice of data set and your results. How does the MLPRegressor differ from the MLPClassifier? What are their respective uses? What lessons from above did you use?

** Your discussion goes here **

### 3.2 (10%) - Other MLP Hyperparameters 
With the same data set, you may (not required) experiment with some of the hyperparameters you already did above (LR, hidden nodes, momentum, validation set parameters, regularization).  But for sure experiment with and discuss the results of the first two hyperparameters below (activation functions and multiple hidden layers).  We encourage you to experiment briefly with the others but they are not required. 

- different hidden layer activation functions (tanh, relu in addition to logistic) - Note that Sklearn does not currently let you choose the output layer activation function.  It is automatically softmax for classification and linear for regression.
- more than one hidden layer
- solver - try adam and lbfgs in addition to sgd
- batch size
- learning rate adaptation - this is the schedule parameter which lets LR adapt during learning

In [ ]:
# Run with different hyperparameters

#### Discussion
How do the hyperparameters affect your accuracy? Specifically the number of layers and nodes and activation functions you experimented with. Do the solver, batch size, and learning rate adaptation affect anything?


** Your discussion goes here **

## 4. (Optional 20% extra credit) Code up your own MLP/Backprop learner 
Below is a scaffold you could use if you want. Requirements for this task:
- Your model should support the methods shown in the example scaffold below.
- Ability to create a network structure with at least one hidden layer and an arbitrary number of nodes. You may choose just one non-linear activation function for all hidden and output nodes if you want (e.g. sigmoid activation function where the loss is SSE rather than cross-entropy).
- Random weight initialization with small random weights with 0 mean. Remember that every hidden and output node should have its own bias weight.
- Use stochastic training updates: update weights after each training instance (i.e. not batch)
- Option to include a momentum term
- Your class can inherit from the relevant scikit-learn learners (e.g. data shuffling, etc.), but don't call any of the super methods to accomplish the core methods in the scaffold.
- Run the Iris data set above with your Backprop version. Show and discuss your results and how they compare with the sklearn version.
- Coding MLP is a good experience but is a little more challening than implementing other models so the extra credit points are higher than typical.

*Discuss your results and any differences*

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class MLP(BaseEstimator,ClassifierMixin):

    def __init__(self,lr=.1, momentum=0, shuffle=True,hidden_layer_widths=None):
        """ Initialize class with chosen hyperparameters.

        Args:
            lr (float): A learning rate / step size.
            shuffle(boolean): Whether to shuffle the training data each epoch. DO NOT SHUFFLE for evaluation / debug datasets.
            momentum(float): The momentum coefficent 
        Optional Args (Args we think will make your life easier):
            hidden_layer_widths (list(int)): A list of integers which defines the width of each hidden layer if hidden layer is none do twice as many hidden nodes as input nodes. (and then one more for the bias node)
            For example: input width 1, then hidden layer will be 3 nodes
        Example:
            mlp = MLP(lr=.2,momentum=.5,shuffle=False,hidden_layer_widths = [3,3]),  <--- this will create a model with two hidden layers, both 3 nodes wide
        """
        self.hidden_layer_widths
        self.lr = lr
        self.momentum = momentum
        self.shuffle = shuffle


    def fit(self, X, y, initial_weights=None):
        """ Fit the data; run the algorithm and adjust the weights to find a good solution

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 2D numpy array with the training targets
        Optional Args (Args we think will make your life easier):
            initial_weights (array-like): allows the user to provide initial weights
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        self.weights = self.initialize_weights() if not initial_weights else initial_weights

        return self

    def predict(self, X):
        """ Predict all classes for a dataset X
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        pass

    def initialize_weights(self):
        """ Initialize weights for perceptron. Don't forget the bias!

        Returns:

        """

        return [0]

    def score(self, X, y):
        """ Return accuracy of model on a given dataset. Must implement own score function.

        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 2D numpy array with targets

        Returns:
            score : float
                Mean accuracy of self.predict(X) wrt. y.
        """

        return 0

    ###  Returns the weights. Not required but helpful for debugging
    def get_weights(self):
        pass
